In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

2023-04-21 21:01:31.814380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 21:01:32.350684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.chdir('../../')

In [3]:
from programs import lstm_tool

2023-04-21 21:01:35.020394: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1::].to_numpy()

scaler = MinMaxScaler()
scale_df = scaler.fit_transform(df[:,0:21])

In [5]:
scale_df.dtype

dtype('float64')

In [6]:
x, y = lstm_tool.make_sequence(scale_df, 60)

In [7]:
x.dtype

dtype('float64')

In [8]:
pp = tf.keras.Sequential()
pp.add(tf.keras.layers.LSTM(64, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True, dropout=0.2))
pp.add(tf.keras.layers.LSTM(32, activation='tanh', return_sequences=False))
pp.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))
pp.add(tf.keras.layers.Dense(units=21, activation='linear'))

In [11]:
state_input.dtype

tf.float32

In [10]:
lstm_input = tf.keras.layers.Input(shape=(60, 21))

input_lstm_layer = tf.keras.layers.LSTM(256, activation='tanh', input_shape=(60, 21), kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True)(lstm_input)
hidden_lstm_layer0 = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=False)(input_lstm_layer)
dropout_lstm_layer = tf.keras.layers.Dropout(0.2)(hidden_lstm_layer0)
hidden_lstm_layer1 = tf.keras.layers.Dense(128, activation='relu')(dropout_lstm_layer)
hidden_lstm_layer2 = tf.keras.layers.Dense(128, activation='relu')(hidden_lstm_layer1)

lstm_layer = tf.keras.models.Model(lstm_input, hidden_lstm_layer2)

state_input = tf.keras.layers.Input(shape=(21,))

input_state_layer = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(state_input)
dropout_state_layer = tf.keras.layers.Dropout(0.3)(input_state_layer)
hidden_state_layer0 = tf.keras.layers.Dense(64, activation='relu')(dropout_state_layer)
hidden_state_layer1 = tf.keras.layers.Dense(64, activation='relu')(hidden_state_layer0)

state_layer = tf.keras.models.Model(state_input, hidden_state_layer1)

input_concat_layer = tf.keras.layers.concatenate([lstm_layer.output, state_layer.output])
hidden_concat_layer0 = tf.keras.layers.Dense(32, activation='relu')(input_concat_layer)
dropout_concat_layer = tf.keras.layers.Dropout(0.3)(hidden_concat_layer0)
hidden_concat_layer1 = tf.keras.layers.Dense(32, activation='relu')(dropout_concat_layer)
output_concat_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_concat_layer1)

sp = tf.keras.models.Model([lstm_input, state_input], output_concat_layer)
sp.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

2023-04-21 21:01:52.292431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-21 21:01:52.293368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-21 21:01:52.293989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus